# Демонстрация использования класса `OscillogramFinder`

Этот ноутбук предназначен для демонстрации возможностей класса `OscillogramFinder` из `filesystem.finder`. `OscillogramFinder` используется для обнаружения, копирования и фильтрации файлов осциллограмм различных форматов, включая обработку архивов и предотвращение дубликатов с помощью хеширования.

Будут рассмотрены следующие функции:
- Копирование новых осциллограмм (`copy_new_oscillograms`) с различными параметрами (выбор типов файлов, сохранение структуры директорий, использование хешей).
- Поиск осциллограмм по номерам терминалов на основе JSON-метаданных (`find_terminal_hashes_from_json`).
- Поиск осциллограмм, содержащих каналы тока нулевой последовательности (`find_oscillograms_with_neutral_current`).

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_finder_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import json
import hashlib
import datetime # Not strictly needed for setup but often for timestamps
import zipfile
import py7zr
try:
    import aspose.zip as az # Conditional import
    HAS_ASPOSE_ZIP_NB = True
except ImportError:
    HAS_ASPOSE_ZIP_NB = False
    print("Предупреждение: Библиотека 'aspose.zip' не найдена. Обработка RAR-архивов будет недоступна/ограничена.")

import pandas as pd # For displaying JSONs if desired
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from filesystem.finder import OscillogramFinder, TYPE_OSC
from core.oscillogram import Oscillogram # Needed by finder's neutral current method

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_path = os.path.join(base_sample_data_dir, "comtrade_files")
archive_samples_path = os.path.join(base_sample_data_dir, "archives")
config_samples_path = os.path.join(base_sample_data_dir, "config_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_finder = "temp_finder_notebook_data"
notebook_source_dir_main = os.path.join(notebook_base_temp_dir_finder, "source_main")
# Cоздадим поддиректории в source_main для теста сохранения структуры
notebook_source_subdir1 = os.path.join(notebook_source_dir_main, "subdir1_f")
notebook_source_subdir2 = os.path.join(notebook_source_dir_main, "subdir2_f")
    
notebook_dest_dir_finder = os.path.join(notebook_base_temp_dir_finder, "destination_oscillograms")
notebook_config_dir_finder = os.path.join(notebook_base_temp_dir_finder, "config_finder")

if os.path.exists(notebook_base_temp_dir_finder):
    shutil.rmtree(notebook_base_temp_dir_finder)
os.makedirs(notebook_source_dir_main)
os.makedirs(notebook_source_subdir1)
os.makedirs(notebook_source_subdir2)
os.makedirs(notebook_dest_dir_finder)
os.makedirs(notebook_config_dir_finder)

# --- Функция для копирования с проверкой существования ---
def copy_sample_file(src_full_path, dest_dir_or_file):
    if os.path.exists(src_full_path):
        # Если dest_dir_or_file это директория, копируем файл туда с тем же именем
        if os.path.isdir(dest_dir_or_file):
            shutil.copy(src_full_path, os.path.join(dest_dir_or_file, os.path.basename(src_full_path)))
        else: # Иначе, это полный путь назначения
            os.makedirs(os.path.dirname(dest_dir_or_file), exist_ok=True)
            shutil.copy(src_full_path, dest_dir_or_file)
        return True
    else:
        print(f"ОШИБКА: Исходный файл для теста не найден: {src_full_path}")
        return False
            
all_files_setup_finder = True
    
# --- Наполнение source_main файлами разных типов ---
# COMTRADE
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.cfg"), notebook_source_dir_main)
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.dat"), notebook_source_dir_main)
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.cfg"), os.path.join(notebook_source_subdir1, "sample_A_copy.cfg")) # Копия в subdir1
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.dat"), os.path.join(notebook_source_subdir1, "sample_A_copy.dat"))
# CFF
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "valid_cff_1", "valid_cff_1.cff"), notebook_source_dir_main)
# Bresler (создадим dummy, если нет в sample_data)
dummy_brs_path = os.path.join(comtrade_samples_path, "other_formats", "sample.brs")
if not os.path.exists(dummy_brs_path): 
    os.makedirs(os.path.join(comtrade_samples_path, "other_formats"), exist_ok=True)
    with open(dummy_brs_path, "w") as f: f.write("dummy brs content")
all_files_setup_finder &= copy_sample_file(dummy_brs_path, notebook_source_dir_main)
    
# EKRA (dfr)
dummy_dfr_path = os.path.join(comtrade_samples_path, "other_formats", "sample.dfr")
if not os.path.exists(dummy_dfr_path): 
    with open(dummy_dfr_path, "w") as f: f.write("dummy dfr content")
all_files_setup_finder &= copy_sample_file(dummy_dfr_path, notebook_source_subdir2)

# Non-oscillogram
with open(os.path.join(notebook_source_dir_main, "report.txt"), "w") as f: f.write("some text report")
    
# Файлы для find_oscillograms_with_neutral_current
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "specific_signals_1", "neutral_current.cfg"), notebook_source_dir_main)
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "specific_signals_1", "neutral_current.dat"), notebook_source_dir_main)
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "specific_signals_1", "signals_for_checker.cfg"), os.path.join(notebook_source_dir_main, "no_neutral.cfg")) # Переименуем для теста
all_files_setup_finder &= copy_sample_file(os.path.join(comtrade_samples_path, "specific_signals_1", "signals_for_checker.dat"), os.path.join(notebook_source_dir_main, "no_neutral.dat"))

# Архивы
all_files_setup_finder &= copy_sample_file(os.path.join(archive_samples_path, "archive_A.zip"), notebook_source_dir_main)
all_files_setup_finder &= copy_sample_file(os.path.join(archive_samples_path, "archive_B.7z"), notebook_source_dir_main)
if HAS_ASPOSE_ZIP_NB: # Только если библиотека доступна, копируем/проверяем RAR
     all_files_setup_finder &= copy_sample_file(os.path.join(archive_samples_path, "archive_C.rar"), notebook_source_dir_main) # Предполагается, что test_data_setup создал его
else:
    print("Пропуск копирования RAR архива, т.к. aspose.zip не доступен.")

# JSON для find_terminal_hashes_from_json
src_terminal_json = os.path.join(config_samples_path, "terminal_hashes_input.json")
dest_terminal_json_finder = os.path.join(notebook_config_dir_finder, "terminal_hashes_input.json")
if os.path.exists(src_terminal_json): shutil.copy(src_terminal_json, dest_terminal_json_finder)
else: print(f"ОШИБКА: terminal_hashes_input.json не найден в {config_samples_path}."); all_files_setup_finder = False

# Начальный _hash_table.json для dest_dir_finder
initial_hash_dat_path = os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.dat")
initial_hash_val = ""
if os.path.exists(initial_hash_dat_path):
    with open(initial_hash_dat_path, "rb") as f_hash: initial_hash_val = hashlib.md5(f_hash.read()).hexdigest()
    
initial_hashes_content_finder = { initial_hash_val: ["sample_A.cfg", "already_copied/sample_A.cfg"] } if initial_hash_val else {}
with open(os.path.join(notebook_dest_dir_finder, "_hash_table.json"), "w", encoding="utf-8") as f:
    json.dump(initial_hashes_content_finder, f)

if not all_files_setup_finder:
    print("Не все файлы для демонстрации OscillogramFinder были успешно настроены.")
else:
    print(f"Директория с исходными файлами и архивами для OscillogramFinder: {notebook_source_dir_main}")
    print(f"Директория назначения: {notebook_dest_dir_finder}")

## 1. Инициализация `OscillogramFinder`

In [ ]:
finder = OscillogramFinder(is_print_message=True, show_progress_bars=True)

## 2. Копирование новых осциллограмм (`copy_new_oscillograms`)
Этот метод является основным. Он рекурсивно обходит исходную директорию, находит файлы осциллограмм (включая те, что в архивах), и копирует их в директорию назначения, избегая дубликатов на основе хеш-сумм (если включено).

In [ ]:
print("\n--- Демонстрация 1: Копирование только COMTRADE (.cfg/.dat, .cff) с сохранением структуры ---")
# Используем notebook_dest_dir_finder, который уже содержит _hash_table.json
# Очистим его от предыдущих файлов, кроме _hash_table.json, для чистоты демонстрации этой ячейки
for item in os.listdir(notebook_dest_dir_finder):
    if item != "_hash_table.json":
        item_path = os.path.join(notebook_dest_dir_finder, item)
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
        else:
            os.remove(item_path)

file_types_to_copy_cfg_cff = { type_enum: False for type_enum in TYPE_OSC } 
file_types_to_copy_cfg_cff[TYPE_OSC.COMTRADE_CFG_DAT] = True
file_types_to_copy_cfg_cff[TYPE_OSC.COMTRADE_CFF] = True
file_types_to_copy_cfg_cff[TYPE_OSC.ARCH_ZIP] = True 
file_types_to_copy_cfg_cff[TYPE_OSC.ARCH_7Z] = True
file_types_to_copy_cfg_cff[TYPE_OSC.ARCH_RAR] = True

copied_count1 = finder.copy_new_oscillograms(
    source_dir=notebook_source_dir_main,
    dest_dir=notebook_dest_dir_finder,
    copied_hashes_input=None, # Заставим его загрузить _hash_table.json из notebook_dest_dir_finder
    preserve_dir_structure=True,
    use_hashes=True,
    file_type_flags=file_types_to_copy_cfg_cff,
    max_archive_depth=2,
    show_progress_method=True
)
print(f"Скопировано файлов (попытка 1): {copied_count1}")
print(f"Содержимое директории назначения ({notebook_dest_dir_finder}):")
for root, dirs, files in os.walk(notebook_dest_dir_finder):
    level = root.replace(notebook_dest_dir_finder, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

if os.path.exists(os.path.join(notebook_dest_dir_finder, "_hash_table.json")):
    with open(os.path.join(notebook_dest_dir_finder, "_hash_table.json"), "r") as f_ht:
        print("\nСодержимое _hash_table.json:")
        try: display(pd.DataFrame(list(json.load(f_ht).items()), columns=['Hash', 'RelativePath']))
        except NameError: print(json.dumps(json.load(f_ht), indent=2, ensure_ascii=False))
        
newly_copied_files_run1 = [f for f in os.listdir(notebook_dest_dir_finder) if f.startswith("newly_copied_hashes_")]
if newly_copied_files_run1:
    with open(os.path.join(notebook_dest_dir_finder, newly_copied_files_run1[0]), "r") as f_nht:
        print(f"\nСодержимое {newly_copied_files_run1[0]}:")
        try: display(pd.DataFrame(list(json.load(f_nht).items()), columns=['Hash', 'RelativePath']))
        except NameError: print(json.dumps(json.load(f_nht), indent=2, ensure_ascii=False))

In [ ]:
print("\n--- Демонстрация 2: Копирование всех типов, без сохранения структуры, без начальных хешей ---")
dest_dir_run2_finder = os.path.join(notebook_base_temp_dir_finder, "destination_run2")
if os.path.exists(dest_dir_run2_finder): shutil.rmtree(dest_dir_run2_finder)
os.makedirs(dest_dir_run2_finder)

copied_count2 = finder.copy_new_oscillograms(
    source_dir=notebook_source_dir_main,
    dest_dir=dest_dir_run2_finder,
    copied_hashes_input=None, # Начнем с чистого листа, без _hash_table.json
    preserve_dir_structure=False, 
    use_hashes=True, 
    file_type_flags=None, # Все типы
    max_archive_depth=2,
    show_progress_method=True
)
print(f"Скопировано файлов (попытка 2): {copied_count2}")
print(f"Содержимое директории назначения ({dest_dir_run2_finder}):")
for root, dirs, files in os.walk(dest_dir_run2_finder):
    level = root.replace(dest_dir_run2_finder, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

## 3. Поиск осциллограмм по номерам терминалов (`find_terminal_hashes_from_json`)
Этот статический метод анализирует JSON-файл, который содержит информацию о путях к файлам осциллограмм, и находит хеши, связанные с указанными номерами терминалов (по конвенциям именования файлов или путей).

In [ ]:
# Используем dest_terminal_json_finder, который был скопирован в ячейке настройки
input_meta_json_path_finder = dest_terminal_json_finder 

output_terminal_hashes_path = os.path.join(notebook_config_dir_finder, "terminal_hashes_output.json")
terminals_to_find = [123, 456, 789] 

print(f"\nПоиск хешей для терминалов: {terminals_to_find}")
OscillogramFinder.find_terminal_hashes_from_json(
    input_json_path=input_meta_json_path_finder,
    terminal_numbers_to_find=terminals_to_find,
    output_json_path=output_terminal_hashes_path,
    is_print_message=True
)

if os.path.exists(output_terminal_hashes_path):
    print(f"Содержимое {output_terminal_hashes_path}:")
    with open(output_terminal_hashes_path, "r", encoding="utf-8") as f:
        try: display(pd.DataFrame(list(json.load(f).items()), columns=['Terminal', 'Hashes']))
        except NameError: print(json.dumps(json.load(f), indent=2, ensure_ascii=False))

## 4. Поиск осциллограмм с каналами тока нулевой последовательности (`find_oscillograms_with_neutral_current`)
Этот метод сканирует `.cfg` файлы в указанной директории, загружает их с помощью `Oscillogram` и анализирует имена аналоговых каналов на предмет наличия признаков тока нейтрали (например, "I" и "N" в имени).

In [ ]:
neutral_current_report_path = os.path.join(notebook_config_dir_finder, "neutral_current_report.txt")

print(f"\nПоиск осциллограмм с током нейтрали в {notebook_source_dir_main}")
finder.find_oscillograms_with_neutral_current(
    cfg_directory=notebook_source_dir_main, 
    output_txt_path=neutral_current_report_path,
    show_progress_method=True # или is_print_message, в зависимости от рефакторинга
)

if os.path.exists(neutral_current_report_path):
    print(f"Содержимое отчета о файлах с током нейтрали ({neutral_current_report_path}):")
    with open(neutral_current_report_path, "r", encoding="utf-8") as f:
        print(f.read())

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_finder):
        shutil.rmtree(notebook_base_temp_dir_finder)
        print(f"\nВременная директория {notebook_base_temp_dir_finder} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_finder} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_finder}: {e}")